In [1]:
import os
api_key=os.getenv("OPENAI_API_KEY")

In [3]:
from langchain_community.document_loaders import PyPDFLoader
pdf_filepath = './차세대 한국형 스마트팜 개발.pdf'
loader = PyPDFLoader(pdf_filepath)
data  = loader.load()

In [4]:
from langchain.text_splitter import CharacterTextSplitter
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
data = text_splitter.split_documents(data)

In [6]:
data

[Document(metadata={'producer': 'Skia/PDF m51', 'creator': 'Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/51.0.2704.63 Safari/537.36', 'creationdate': '2018-08-23T06:06:07+00:00', 'moddate': '2018-08-23T06:06:07+00:00', 'source': './차세대 한국형 스마트팜 개발.pdf', 'total_pages': 4, 'page': 0, 'page_label': '1'}, page_content='차세대\xa0한국형스마트팜\xa0개발\n국립농업과학원\xa0농업공학부\xa0스마트팜개발과\xa0이현동\xa0연구관\n한국형\xa0스마트팜\xa0개발\xa0프로젝트\xa0!\n우리나라\xa0농업은\xa0고령화와\xa0높은\xa0노동강도로\xa0인해\xa0청년인구의\xa0유입이\xa0어려운\xa0구조로\xa0타\xa0분야보다\xa0생산인구\xa0절벽화가\xa0더욱\n가시화되고\xa0있다.\xa0이에\xa0대한\xa0대안으로\xa0제시되고\xa0있는\xa0스마트팜은\xa0사물인터넷,\xa0빅데이터,\xa0인공지능,\xa0로봇\xa0등을\xa0활용하여\n농산물의\xa0생육환경을\xa0최적상태로\xa0관리하고\xa0노동력\xa0절감과\xa0생산성\xa0향상을\xa0구현하는\xa0효율적인\xa0농업형태이다.\xa0\n차세대\xa0한국형\xa0스마트팜\xa0기술개발\xa0프로젝트는\xa04기관\xa019개\xa0전담부서가\xa0협업하여\xa0핵심\xa0요소\xa0및\xa0원천\xa0기반기술의\xa0확보를\xa0위해\n연구\xa0역량을\xa0집중하고\xa0있고\xa0국내\xa0농업여건에\xa0적합하게\xa0기술수준별로\xa0스마트팜\xa0모델을\xa03가지\xa0단계로\xa0구분하여\xa0개발을\n추진하고\xa0있다.\xa0단계별\xa0스마트팜

In [5]:
from langchain.embeddings.openai import OpenAIEmbeddings
embeddings = OpenAIEmbeddings(model="text-embedding-ada-002", api_key=api_key)

/var/folders/fs/xj6_f58s74z5txbxnt876g100000gn/T/ipykernel_51693/1332705152.py:2: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
  embeddings = OpenAIEmbeddings(model="text-embedding-ada-002", api_key=api_key)


In [7]:
from langchain.vectorstores import FAISS
vectorstore = FAISS.from_documents(data, embeddings)

In [8]:
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(model="gpt-3.5-turbo", api_key=api_key)

/var/folders/fs/xj6_f58s74z5txbxnt876g100000gn/T/ipykernel_51693/1264873333.py:3: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm = ChatOpenAI(model="gpt-3.5-turbo", api_key=api_key)


In [9]:
from langchain.memory import ConversationBufferMemory
memory = ConversationBufferMemory(memory_key='chat_history', return_messages=True)

/var/folders/fs/xj6_f58s74z5txbxnt876g100000gn/T/ipykernel_51693/952344495.py:2: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory(memory_key='chat_history', return_messages=True)


In [10]:
from langchain.chains import ConversationalRetrievalChain
conversation_chain = ConversationalRetrievalChain.from_llm(
    llm=llm,
    chain_type="stuff",
    retriever=vectorstore.as_retriever(),
    memory=memory
)

In [11]:
query = "한국형 스마트팜이란?"
result = conversation_chain({"question": query})
answer = result["answer"]
answer

/var/folders/fs/xj6_f58s74z5txbxnt876g100000gn/T/ipykernel_51693/139330662.py:2: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  result = conversation_chain({"question": query})


'한국형 스마트팜은 사물인터넷, 빅데이터, 인공지능, 로봇 등의 첨단 기술을 활용하여 농산물의 생육환경을 최적화하고 노동력을 절감하며 생산성을 향상시키는 효율적인 농업 형태를 일컫는다. 농업의 미래를 대비하기 위해 발전하는 스마트 농업 기술 중 한국에서 개발된 형태를 의미합니다.'

In [12]:
query = "그것에 필요한 기술은?"
result = conversation_chain({"question": query})
answer = result["answer"]
answer

'농업 분야에서 필요한 기술은 사물인터넷, 빅데이터, 인공지능, 로봇 등을 활용하여 스마트 기기를 농사에 효과적으로 접목할 수 있는 ICT분야 연구 및 개발이 중요합니다. 특히, 스마트팜에서 빅데이터를 활용한 생산성 향상, 에너지 최적화 및 자동화를 포함한 스마트팜 통합시스템, 그리고 스마트팜의 ICT 기기 규격화 및 표준화가 필요한 중요한 기술입니다.'